In [1]:
"""
CheXNet applied to chest X-ray images classifying pneumonia vs non-pneumonia.
"""

import tensorflow.python.keras.backend as K
import numpy as np
import tensorflow as tf
from loss import WeightedCrossEntropyBinaryLoss
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Dense
from keras.models import Model
import prepare_data
import os

c:\Users\marta\anaconda3\envs\iannwtf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# input_path = '../input/chest_xray/chest_xray/'
input_path = 'C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/chest_xray/'

# Checkpoint file path
checkpoint_filepath = 'C:/Users/marta/OneDrive/Desktop/Osnabruck/ImplementingANNswithTensorFlow/FinalProject/Checkpoint/weights.hdf5' 

# Train data path
train_data_path = input_path+'train'

# Val data path
val_data_path = input_path+'test'

# Hyperparameters
img_dims = 224
n_epochs = 10
batch_size = 16 # since it's a binary classification task
val_batch_size = 64
output_size = 1

# Getting the data
train_gen, test_gen, test_data, test_labels = prepare_data.prepare_data(img_dims, batch_size)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [26]:
class CheXNet(tf.keras.Model):
  """
  The model using modified DenseNet121.
  """

  def __init__ (self):
    """
    The constructor initiates the layers and their activation functions....
    """
    super().__init__()

    # Following will be set by get_data_stats() based on the dataset
    self.zero_weight = None
    self.one_weight = None
    self.train_steps = None
    self.val_steps = None

    # get_model() will initialize this to DenseNet121 model
    self.model = None
  
  def get_data_stats(self, train_data_path, val_data_path):
    """
    Computes normal Vs Pneumonia class distribution.
    :param train_data_path: path to training data. Samples os each class should be stored in separate folders
    :param val_data_path: path to validation data. Samples os each class should be stored in separate folders
    """

    # Count images in each class
    n_normal = len(os.listdir(train_data_path + '/NORMAL'))
    n_infect = len(os.listdir(train_data_path + '/PNEUMONIA'))

    # compute class distribution
    self.w_class1 = float(n_normal)/(n_normal+n_infect)
    self.w_class0 = float(n_infect)/(n_normal+n_infect)

    # For convenience at train time, compute number of steps required to complete an epoch
    n_normal_val = len(os.listdir(val_data_path + '/NORMAL'))
    n_infect_val = len(os.listdir(val_data_path + '/PNEUMONIA'))

    self.train_steps = ((n_normal+n_infect) // batch_size) + 1
    self.val_steps = ((n_normal_val+n_infect_val) // val_batch_size) + 1  


  def get_model(self):
    # DenseNet121 expects number of channels to be 3
    input = Input(shape=(img_dims, img_dims, 3))

    # using pretrained DenseNet121 as the foundation of the model
    base_model = tf.keras.applications.densenet.DenseNet121(include_top=False, weights='imagenet', input_tensor=input,
                                                            input_shape=(img_dims, img_dims, 3))
    
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.layers[-1].output)
    x = tf.keras.layers.Dense(output_size, activation=tf.nn.sigmoid)

    self.model = Model()

    # Using weighted binary loss has been suggested in the paper
    loss = WeightedCrossEntropyBinaryLoss(self.zero_weight, self.one_weight)

    # Compile the model
    self.model.compile(optimizer=tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999),
                       loss=loss.weighted_binary_crossentropy, metrics=['accuracy'])
    
    return self.model

# Instantiate the CheXNet model
chexnet = CheXNet()

# Compute normal Vs Pneumonia class distribution
chexnet.get_data_stats(train_data_path, val_data_path)

# Create and compile the DenseNet121 model
model = chexnet.get_model()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [32]:
print(train_gen)

In [30]:
# #Fitting the model
history = model.fit_generator(
    train_gen, epochs=n_epochs, validation_data=tuple(test_gen), callbacks=[reduce_lr, model_checkpoint],
    steps_per_epoch=chexnet.train_steps, validation_steps=chexnet.val_steps)


# Load the model with the lowest validation loss
model.load_weights(checkpoint_filepath)

model.summary()


KeyboardInterrupt: 